In [87]:
from keras.models import Model
from keras.layers import *
from keras.datasets import mnist
from matplotlib import pyplot as plt

#prepare data
from keras.utils import np_utils 
import numpy as np
import os
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train =x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_test/=255
x_train/=255
y_train=np_utils.to_categorical(y_train,10)
y_test= np_utils.to_categorical(y_test,10)

#num of choice per parameter
optimize=7
activate=4
layer=3
conv=3

#num of population,generation
pop=12
gen=10

#create first parameters
fake_opti=np.random.randint(0,optimize,pop)
fake_acti=np.random.randint(0,activate,pop)
fake_laye=np.random.randint(0,layer,pop)
fake_conv=np.random.randint(0,conv,pop)



In [88]:
print(fake_acti)
print(fake_conv)
print(fake_opti)
print(fake_laye)

[0 0 0 2 3 0 0 3 3 2 3 0]
[2 1 1 2 0 2 2 2 1 2 2 0]
[3 4 2 4 0 4 5 3 4 3 0 3]
[1 1 0 2 1 0 2 0 1 0 1 1]


In [89]:
def mutate():
    for j in range(pop-4):
        
        #make 3 clones each
        i=j//4
        for h in range(4):
            new_pop[h][j+4]=new_pop[h][i]
        
        #mutate
        if np.random.rand(1)<0.25:
            new_pop[0][j+4]=np.random.randint(0,activate)
        if np.random.rand(1)<0.25:
            rand1=np.random.randint(0,4)
            rand2=np.random.randint(0,4)
            rand3=np.random.randint(0,4)
            new_pop[1][j+4]=new_pop[1][rand1]+0.75*(new_pop[1][rand2]-new_pop[1][rand3])
        if np.random.rand(1)<0.25:
            new_pop[2][j+4]=np.random.randint(0,optimize)
        if np.random.rand(1)<0.25:
            rand1=np.random.randint(0,4)
            rand2=np.random.randint(0,4)
            rand3=np.random.randint(0,4)
            new_pop[3][j+4]=new_pop[3][rand1]+0.75*(new_pop[3][rand2]-new_pop[3][rand3])
    return

In [90]:
def keep(m):
    new_opti.append(fake_opti[m])
    new_conv.append(fake_conv[m])
    new_acti.append(fake_acti[m])
    new_laye.append(fake_laye[m])
    return


In [91]:
#determine best combinantion of parameters
for i in range(gen):
    print(fake_acti)
    print(fake_conv)
    print(fake_opti)
    print(fake_laye)
    
    #change parameter to useful values
    real_acti=[]
    for a in range(pop):
        b=fake_acti[a]
        if b==0:
            c='elu'
        if b==1:
            c='relu'
        if b==2:
            c='tanh'
        if b==3:
            c='sigmoid'
        real_acti.insert(a,c)
        
    fake_conv=fake_conv-fake_conv%1
    real_conv=16*2**fake_conv
    
    real_opti=[]
    for d in range(pop):
        e=fake_opti[d]
        if e==0:
            f='rmsprop'
        if e==1:
            f='adam'
        if e==2:
            f='sgd'
        if e==3:
            f='adagrad'
        if e==4:
            f='adadelta'
        if e==5:
            f='adamax'
        if e==6:
            
            f='nadam'
        real_opti.insert(d,f)
    
    real_laye=fake_laye-fake_laye%1

    print(real_acti)
    print(real_opti)
    
    #evaluate each model in generation
    score_list=[]
    print('\ngen.',i+1,'\n')
    for j in range(pop):
        input_layer= Input(shape=(28,28,1))
        conv=Conv2D(real_conv[j],(3,3),activation=real_acti[j])(input_layer)
        
        #add extra layers
        for k in range(real_laye[j]):
            conv=Conv2D(real_conv[j],(3,3),activation=real_acti[j])(conv)
        maxpool1= MaxPooling2D(pool_size=(2,2))(conv)
        flat1=Flatten()(maxpool1)
        dense1= Dense(128,activation=real_acti[j])(flat1)
        output=Dense(10,activation='softmax')(dense1)
        model=Model(inputs=input_layer,outputs=output)
        model.compile(loss='categorical_crossentropy',optimizer=real_opti[j],metrics=["accuracy"])
        print('\nnum.',j+1,'\n')
        print(model.summary())
        print(model.output_shape)
        model.fit(x_train,y_train,batch_size=32,epochs=1,verbose=1)
        scores=model.evaluate(x_test,y_test,verbose=1)
        print("loss",scores[0])
        print("accuracy",scores[1])
        score_list.append(scores[1])
        
    #sort scores    
    A=np.array(score_list)
    B=A.argsort()[::-1]
    print(B[0])
    print(score_list[B[0]])
    
    #no more need for creating new parameters
    if i==gen-1:
        break
        
   #keep selected parameters
    new_opti=[]
    new_conv=[]
    new_acti=[]
    new_laye=[]
    
    #keep 3 top 
    for l in range(3):
        m=B[l]
        keep(m)
    
    #keep 1 random    
    p=np.random.randint(pop-3)
    m=B[p+3]
    keep(m)
    
    #new population arranged for mutation
    new_pop=[[0 for h in range(pop)]for e in range(4)]
    for g in range(4):
        new_pop[0][g]=new_acti[g]
        new_pop[1][g]=new_conv[g]
        new_pop[2][g]=new_opti[g]
        new_pop[3][g]=new_laye[g]
    
    #mutate
    mutate()
    
    #introduce new parameters
    for f in range(pop):
        fake_acti[f]=new_pop[0][f]
        fake_conv[f]=new_pop[1][f]
        fake_opti[f]=new_pop[2][f]
        fake_laye[f]=new_pop[3][f]
    
    
        


[0 0 0 2 3 0 0 3 3 2 3 0]
[2 1 1 2 0 2 2 2 1 2 2 0]
[3 4 2 4 0 4 5 3 4 3 0 3]
[1 1 0 2 1 0 2 0 1 0 1 1]
['elu', 'elu', 'elu', 'tanh', 'sigmoid', 'elu', 'elu', 'sigmoid', 'sigmoid', 'tanh', 'sigmoid', 'elu']
['adagrad', 'adadelta', 'sgd', 'adadelta', 'rmsprop', 'adadelta', 'adamax', 'adagrad', 'adadelta', 'adagrad', 'rmsprop', 'adagrad']

gen. 1 


num. 1 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_76 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 12, 12, 64)        0         
_______________________________________________

dense_161 (Dense)            (None, 128)               1384576   
_________________________________________________________________
dense_162 (Dense)            (None, 10)                1290      
Total params: 1,386,506
Trainable params: 1,386,506
Non-trainable params: 0
_________________________________________________________________
None
(None, 10)
Epoch 1/1
10000/10000 [==============================] - 6s 593us/step
loss 0.07870280007589608
accuracy 0.9731

num. 7 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_82 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_159 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_160 (Conv2D)          (None, 24, 24, 64)        36928     
____________________________________________________________

Epoch 1/1
10000/10000 [==============================] - 5s 459us/step
loss 0.060991909652017055
accuracy 0.9793
6
0.9859
[0 0 0 3 0 0 3 0 0 0 0 0]
[2 1 2 2 2 2 2 2 1 1 1 2]
[5 4 3 3 5 5 5 5 4 4 1 4]
[2 1 1 0 2 0 2 2 1 1 1 1]
['elu', 'elu', 'elu', 'sigmoid', 'elu', 'elu', 'sigmoid', 'elu', 'elu', 'elu', 'elu', 'elu']
['adamax', 'adadelta', 'adagrad', 'adagrad', 'adamax', 'adamax', 'adamax', 'adamax', 'adadelta', 'adadelta', 'adam', 'adadelta']

gen. 2 


num. 1 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_88 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_170 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_171 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
conv

Epoch 1/1
10000/10000 [==============================] - 6s 613us/step
loss 0.08799673299714923
accuracy 0.9724

num. 7 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_94 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_182 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_183 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
conv2d_184 (Conv2D)          (None, 22, 22, 64)        36928     
_________________________________________________________________
max_pooling2d_94 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_94 (Flatten)         (None, 7744)              0         
____________________

Epoch 1/1
10000/10000 [==============================] - 17s 2ms/step
loss 0.06416798925765324
accuracy 0.9787
4
0.9851
[0 0 0 0 0 0 0 0 1 0 0 0]
[2 1 1 1 2 2 2 2 1 1 1 0]
[5 4 4 1 5 5 5 5 4 4 4 4]
[2 1 1 1 2 1 1 1 1 1 1 1]
['elu', 'elu', 'elu', 'elu', 'elu', 'elu', 'elu', 'elu', 'relu', 'elu', 'elu', 'elu']
['adamax', 'adadelta', 'adadelta', 'adam', 'adamax', 'adamax', 'adamax', 'adamax', 'adadelta', 'adadelta', 'adadelta', 'adadelta']

gen. 3 


num. 1 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_100 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_196 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_197 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
conv2d_198 

...

Epoch 1/1
10000/10000 [==============================] - 15s 2ms/step
loss 0.03773625935288146
accuracy 0.9875

num. 12 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_171 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_340 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_341 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_171 (MaxPoolin (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_171 (Flatten)        (None, 4608)              0         
_________________________________________________________________
dense_341 (Dense)            (None, 128)               589952    
____________________

Epoch 1/1
10000/10000 [==============================] - 15s 2ms/step
loss 0.036166527024284
accuracy 0.988

num. 6 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_177 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_352 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_353 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_177 (MaxPoolin (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_177 (Flatten)        (None, 9216)              0         
_________________________________________________________________
dense_353 (Dense)            (None, 128)               1179776   
________________________

Epoch 1/1
10000/10000 [==============================] - 10s 1ms/step
loss 0.17582755978107453
accuracy 0.9454

num. 12 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_183 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_364 (Conv2D)          (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_365 (Conv2D)          (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_183 (MaxPoolin (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_183 (Flatten)        (None, 4608)              0         
_________________________________________________________________
dense_365 (Dense)            (None, 128)               589952    
____________________

Epoch 1/1
10000/10000 [==============================] - 19s 2ms/step
loss 0.049484593592211606
accuracy 0.9836

num. 6 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_189 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_376 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_377 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_189 (MaxPoolin (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_189 (Flatten)        (None, 9216)              0         
_________________________________________________________________
dense_377 (Dense)            (None, 128)               1179776   
____________________

Epoch 1/1
10000/10000 [==============================] - 16s 2ms/step
loss 0.06488045976040885
accuracy 0.979

num. 12 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_195 (InputLayer)       (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_388 (Conv2D)          (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_389 (Conv2D)          (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_195 (MaxPoolin (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_195 (Flatten)        (None, 9216)              0         
_________________________________________________________________
dense_389 (Dense)            (None, 128)               1179776   
_____________________

In [83]:
print(score_list,B)


print('final accuracy=',score_list[B[0]])
    

[0.9574, 0.983, 0.101, 0.9852, 0.976] [3 1 4 0 2]
final accuracy= 0.9852


In [84]:

print(fake_acti)
print(real_acti)
print(fake_conv)
print(fake_opti)
print(fake_laye)

[3 0 3 1 0]
['sigmoid', 'elu', 'sigmoid', 'relu', 'elu']
[1 2 2 2 0]
[1 0 3 0 6]
[2 2 0 2 0]


In [85]:
    new_opti=[]
    new_conv=[]
    new_acti=[]
    new_laye=[]
    
    #keep 3 top 
    for l in range(3):
        m=B[l]
        keep(m)
    
    #keep 1 random    
    p=np.random.randint(pop-3)
    m=B[p+3]
    keep(m)
    
    #new population arranged for mutation
    new_pop=[[0 for h in range(pop)]for e in range(4)]
    for g in range(4):
        new_pop[0][g]=new_acti[g]
        new_pop[1][g]=new_conv[g]
        new_pop[2][g]=new_opti[g]
        new_pop[3][g]=new_laye[g]
    
    #mutate
    mutate()
    
    #introduce new parameters
    for f in range(pop):
        fake_acti[f]=new_pop[0][f]
        fake_conv[f]=new_pop[1][f]
        fake_opti[f]=new_pop[2][f]
        fake_laye[f]=new_pop[3][f]
    
    
        

In [86]:
print(new_pop)

[[1, 0, 0, 0, 0], [2, 2, 0, 0, 0.5], [0, 0, 6, 6, 0], [2, 2, 0, 0, 2]]
